In [2]:
% pylab inline
figsize(10, 8)

import sys
sys.path.append("D:/ds/code")

Populating the interactive namespace from numpy and matplotlib


In [ ]:
from pymongo import MongoClient
from Polygon import Polygon
from Polygon import Error as PolygonError

from geopy.distance import vincenty

# connect to mongodb
client = MongoClient("localhost", 27017)
db = client["twitter"]

# get the tweet cursor
cursor = db.geotweets.find()

real_loc = []
inf_loc  = []
inf_poly = []
distances = []
weights = []
insides = []
usernames = []
areas = []

totalcount = 0
count = 0
nocount = 0

for doc in cursor:
    totalcount += 1
    if doc["geo"] is None:
        continue

    # get the real point
    point = (doc["geo"]["coordinates"][1], doc["geo"]["coordinates"][0])
    # get the poly
    try:
        poly = eval(doc["locinf"]["mi"]["test"]["poly"])
    except KeyError:
        print("Poly not in tweet doc")
        nocount += 1
        continue
        
    centres = []
    
    p = Polygon()
    for i in poly:
        p.addContour(i)

    try:
        center = p.center()
    except PolygonError:
        nocount += 1
        continue

    inside = p.isInside(point[0], point[1])
    distance = vincenty((point[1], point[0]), (center[1], center[0])).km
    
    areas.append(p.area())
    real_loc.append(point)
    inf_loc.append(center)
    inf_poly.append(poly)
    weights.append(doc["locinf"]["mi"]["test"]["weight"])
    distances.append(distance)
    insides.append(inside)
    usernames.append(doc["user"]["screen_name"])

    count += 1
    if count % 1000 == 0:
        print("Processed", count)
    #print("Inside:", ('Y' if inside else 'N'), "- Distance:", distance)

print("Mean distance from polygon center:", sum(distances) / len(distances))
print("Fraction within polygon:", sum(insides) / len(insides))
print("Classified:", (count-nocount) / count)

In [ ]:
# get the count for each user
from collections import defaultdict

counts = defaultdict(int)
for i in usernames:
    counts[i] += 1

user_dict = counts
items = counts.items()
items = sorted(items, key=lambda x: -x[1])

In [ ]:
f_real = 0
f_inf = 1
f_dist = 2
f_weight = 3
f_area = 4
f_inside = 5
f_username = 6
f_usercount = 7

dcounts = [counts[c] for c in usernames]
data = asarray([real_loc, inf_loc, distances, weights, areas, insides, usernames, dcounts]).T
data

In [ ]:
def dta(limit):
    return data[data[:,f_usercount] < limit]

xlabel('Error distance (km)')
ylabel('Count')
h = hist(data[:,f_dist], bins=100)

In [ ]:
def stats(d, title):
    print('====', title, '====')
    print("Max error distance:    %.2fkm" % max(d[:,f_dist]))
    print("Min error distance:    %.2fkm" % min(d[:,f_dist]))
    print("Mean error distance:   %.2fkm" % mean(d[:,f_dist]))
    print("Median error distance: %.2fkm" % median(d[:,f_dist]))
    print("")
    
stats(data, "All tweets")
stats(dta(2000), "Less than 2000 tweets")
stats(dta(100), "Less than 100 tweets")

In [ ]:
def acck(d, k):
    return sum(d <= k) / len(d)

figsize(10, 8)

tests = [10000, 2000, 1000, 50, 2]
for i in tests:
    d = dta(i)[:,f_dist]
    plot([acck(d, k) for k in range(0, 10000, 10)], label='Max %i tweets' % i)

title('Accuracy @ k distance')
xlabel('k error (km)')
ylabel('Fraction of tweets')
legend(loc=4)

In [ ]:
title('Error distance for users with less than 2000 tweets in period')
xlabel('Error distance (km)')
ylabel('Count')
h = hist(dta(2000)[:,f_dist], bins=100)

In [ ]:
from implementation.multiind.polystacker import plot_area

inf_poly_weights = []
for i in range(len(weights)):
    if user_dict[usernames[i]] > 50:
        continue
        
    if inf_poly[i] is not None and weights[i] is not None:
        inf_poly_weights.append([(inf_poly[i][x], weights[i]) for x in range(len(inf_poly[i]))])

res = plot_area(inf_poly_weights)

In [9]:
a = [1,2,3]
a.append(4)
a[1:]

[2, 3, 4]

In [ ]:
res_m = res[0]# clip(res[0], 0, 1)

figsize(20, 6)
imshow(res_m, interpolation='none')
colorbar(orientation='vertical')

In [ ]:
def acck(d, k):
    return sum(d <= k) / len(d)

figsize(10, 8)

tests = [10000, 2000, 1000, 50, 2]
for i in tests:
    dis = distanceslimited(i)
    plot([acck(dis, k) for k in range(0, 10000, 10)], label='Max %i tweets' % i)

legend(loc=4)

In [ ]:
from mpl_toolkits.basemap import Basemap

fig = figure(figsize=(14,7))
ax = fig.add_subplot(111, frame_on=False)
mp = Basemap(projection='robin', lat_0=0, lon_0=0,
              resolution='l', area_thresh=1000.0)

for p in inf_loc:
    if p is None: 
        continue
    x,y = mp(p[0], p[1])
    mp.plot(x, y, '.2', markersize=2, color='green', alpha=0.4)

#mp.drawcoastlines()
#mp.drawcountries()
mp.fillcontinents(color='#CCCCCC')#, zorder=0)
mp.drawmapboundary()

plt.savefig('map-points.png', dpi=200, alpha=True)

In [ ]:
fig = figure(figsize=(14,7))
ax = fig.add_subplot(111, frame_on=False)
mp = Basemap(projection='robin', lat_0=0, lon_0=0,
              resolution='l', area_thresh=1000.0)

for p in range(len(inf_loc)):
    if real_loc[p] is None or inf_loc[p] is None: 
        continue
        
    x1, y1 = mp(real_loc[p][0], real_loc[p][1])
    x2, y2 = mp(inf_loc[p][0], inf_loc[p][1])
    
    mp.plot((x1, x2), (y1, y2), 'b', color='green', alpha=0.1)

#mp.drawcoastlines()
#mp.drawcountries()
mp.fillcontinents(color='#CCCCCC')#, zorder=0)
mp.drawmapboundary()

plt.savefig('map-lines2.png', dpi=400, alpha=True)

In [ ]:
from mpl_toolkits.basemap import Basemap

fig = figure(figsize=(14,7))
ax = fig.add_subplot(111, frame_on=False)
mp = Basemap(projection='robin', lat_0=0, lon_0=0,
              resolution='l', area_thresh=1000.0)

for p in real_loc:
    if p is None: 
        continue
    x,y = mp(p[0], p[1])
    mp.plot(x, y, 'o', markersize=2, color='green', alpha=0.4)

#mp.drawcoastlines()
#mp.drawcountries()
mp.fillcontinents(color='#CCCCCC')#, zorder=0)
mp.drawmapboundary()

plt.savefig('map-points.png', dpi=200, alpha=True)